In [1]:
!pip install ultralytics
!pip install tqdm
!pip install pandas
!pip install pillow

In [2]:
from ultralytics import YOLO
import ultralytics
from tqdm import tqdm
import os
import pandas as pd
from PIL import Image
import sys

In [3]:
# Load a model
model = YOLO("yolov8s.yaml")  # build a new model from scratch
model = YOLO("models/YOLOv8s.pt")  # load a pretrained model (recommended for training)

In [4]:
# Define the path
path = 'images/original_images'

# List all files in the directory
file_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

## Original Prediction

In [5]:
results = []

for i in tqdm(range(len(file_list))):
    res = model.predict(file_list[i], verbose=False)
    results.append(res[0])

100%|██████████| 10/10 [00:01<00:00,  5.63it/s]


In [6]:
# Initialize an empty list to store the data
data = []
for i in range(len(results)):
    try:
        # Extract bounding boxes for each result
        boxes = results[i].boxes.xyxy.tolist()
        classes = results[i].boxes.cls.tolist()
        Name = file_list[i].split('/')[-1]

        # Check if there are no boxes detected
        if not boxes:
            data.append([Name+'_0', 0, 0, 0, 0])
        else:
            # Append each box with the associated file name to the data list
            cnt = 0
            for box, cls in zip(boxes, classes):
                x_min, y_min, x_max, y_max = box
                data.append([Name+'_'+str(cnt), x_min, y_min, x_max, y_max, cls])
                cnt+=1
                
    except Exception as e:
        print(i)
        raise e

In [7]:
df_pred = pd.DataFrame(data, columns=['Name', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])

## Original Ground Truth

In [8]:
# Define the path to the directory containing the text files

# Initialize an empty list to store the ground truth data
ground_truth_txt = []
txt_path = 'ground_truth_files/'
# Iterate through each file in the directory
for f in file_list:
    filename = f.split('/')[-1].split('.')[0] + '.txt'
    file_path = os.path.join(txt_path, filename)
    with open(file_path, 'r') as file:
        content = file.read()
        ground_truth_txt.append(content)

In [9]:
ground_truth = []

for gt in ground_truth_txt:
    ground_truth.append([float(num) for num in gt.split() if num not in ['\n']])

In [10]:
for i in range(len(ground_truth)):
    image = Image.open(file_list[i])
    width = image.width
    height = image.height
    for j in range(len(ground_truth[i])):
        if j % 5 == 0:
            continue
        if j % 2 == 0:
            ground_truth[i][j] *= width
        else:
            ground_truth[i][j] *= height

In [11]:
def convert_to_corners(center_x, center_y, width, height):
    # Calculate top-left corner (xmin, ymin)
    xmin = center_x - (width / 2)
    ymin = center_y - (height / 2)

    # Calculate bottom-right corner (xmax, ymax)
    xmax = center_x + (width / 2)
    ymax = center_y + (height / 2)

    return xmin, ymin, xmax, ymax

In [12]:
for i in range(len(ground_truth)):
        length = len(ground_truth[i])
        cnt = 0
        while(length):
                ground_truth[i][cnt*5+1], ground_truth[i][cnt*5+2], ground_truth[i][cnt*5+3], ground_truth[i][cnt*5+4] = convert_to_corners(ground_truth[i][cnt*5+1], ground_truth[i][cnt*5+2], ground_truth[i][cnt*5+3], ground_truth[i][cnt*5+4])
                length -= 5
                cnt+=1

In [13]:
# Initialize an empty list to store the data
data = []

for i in range(len(ground_truth)):
    try:
        Name = file_list[i].split('/')[-1]  # Assuming there's a filename attribute
        # Initialize an empty list to hold the slices
        slices = []

        # Iterate through the numbers in steps of 5
        for j in range(0, len(ground_truth[i]), 5):
            # Create a slice of 4 elements
            slice = ground_truth[i][j:j+5]
            # Add the slice to the list of slices
            slices.append(slice)
            
            
        cnt = 0
        for box in slices:
            cls, x_min, y_min, x_max, y_max = box
            data.append([Name+'_'+str(cnt), x_min, y_min, x_max, y_max, cls])
            cnt+=1

    except Exception as e:
        # print(slices)
        # print(box)
        print(i)
        raise e

In [14]:
df_gt = pd.DataFrame(data, columns=['Name', 'x_min', 'y_min', 'x_max', 'y_max', 'class'])

## Whatsapp Ground Truth

In [15]:
ground_truth = []

for gt in ground_truth_txt:
    ground_truth.append([float(num) for num in gt.split() if num not in ['\n']])

In [16]:
file_list[0].replace('original_images','whatsapp_images')

'images/whatsapp_images/dc7d6c6fa1fdde25e0aa64a1f6fd594a.png'

In [17]:
for i in range(len(ground_truth)):
    image = Image.open(file_list[i].replace('original_images','whatsapp_images'))
    width = image.width
    height = image.height
    for j in range(len(ground_truth[i])):
        if j % 5 == 0:
            continue
        if j % 2 == 0:
            ground_truth[i][j] *= width
        else:
            ground_truth[i][j] *= height

In [18]:
for i in range(len(ground_truth)):
        length = len(ground_truth[i])
        cnt = 0
        while(length):
                ground_truth[i][cnt*5+1], ground_truth[i][cnt*5+2], ground_truth[i][cnt*5+3], ground_truth[i][cnt*5+4] = convert_to_corners(ground_truth[i][cnt*5+1], ground_truth[i][cnt*5+2], ground_truth[i][cnt*5+3], ground_truth[i][cnt*5+4])
                length -= 5
                cnt+=1

In [19]:
# Initialize an empty list to store the data
data = []

for i in range(len(ground_truth)):
    try:
        Name = file_list[i].split('/')[-1]  # Assuming there's a filename attribute
        # Initialize an empty list to hold the slices
        slices = []

        # Iterate through the numbers in steps of 5
        for j in range(0, len(ground_truth[i]), 5):
            # Create a slice of 4 elements
            slice = ground_truth[i][j:j+5]
            # Add the slice to the list of slices
            slices.append(slice)
            
            
        cnt = 0
        for box in slices:
            cls, x_min, y_min, x_max, y_max = box
            data.append([Name+'_'+str(cnt), x_min, y_min, x_max, y_max, cls])
            cnt+=1

    except Exception as e:
        # print(slices)
        # print(box)
        print(i)
        raise e

In [20]:
df_gt_wp = pd.DataFrame(data, columns=['Name', 'x_min', 'y_min', 'x_max', 'y_max', 'class'])

## WP Prediction

In [21]:
wp_images_path = 'images/whatsapp_images'

In [22]:
wp_pred_Names = []
for filename in os.listdir(wp_images_path):
    wp_pred_Names.append(filename)

In [23]:
# Define the path

# List all files in the directory
wp_file_list = [os.path.join(wp_images_path, f) for f in os.listdir(wp_images_path) if os.path.isfile(os.path.join(wp_images_path, f))]

In [24]:
wp_file_list

['images/whatsapp_images/dc7d6c6fa1fdde25e0aa64a1f6fd594a.png',
 'images/whatsapp_images/e529c5465cafd94ee9a3b38f7267523a.png',
 'images/whatsapp_images/da95c308ecf6b869be4930aa124c0d7e.png',
 'images/whatsapp_images/fb16b22a22ab5aa31e8de22118a8183c.png',
 'images/whatsapp_images/d79e29f792f1225d45994d75d624f8d1.png',
 'images/whatsapp_images/e7ae00eef602436f48a44b36dc79b383.png',
 'images/whatsapp_images/ecef71a87df9aee5cca62966f95d9f26.png',
 'images/whatsapp_images/f51c1a48919f8b36116ed4aa799dcb23.png',
 'images/whatsapp_images/fd079817727211a09fd6b0415463f23d.png',
 'images/whatsapp_images/e67e4cab40626c54b8fc20ce6e9e4273.png']

In [25]:
wp_results = []

for i in tqdm(range(len(wp_file_list))):
    res = model.predict(wp_file_list[i], verbose=False)
    wp_results.append(res[0])

100%|██████████| 10/10 [00:00<00:00, 42.83it/s]


In [26]:
# Initialize an empty list to store the data
data = []
cnt1 = 0
for i in range(len(wp_results)):
    try:
        # Extract bounding boxes for each result
        boxes = wp_results[i].boxes.xyxy.tolist()
        classes = wp_results[i].boxes.cls.tolist()
        Name = wp_file_list[i].split('/')[-1]  # Assuming there's a filename attribute
        
        cnt1+=1
            # Check if there are no boxes detected
        if not boxes:
            data.append([Name+'_0', 0, 0, 0, 0])
        else:
            # Append each box with the associated file name to the data list
            cnt = 0
            for cls, box in zip(classes,boxes):
                x_min, y_min, x_max, y_max = box
                data.append([Name+'_'+str(cnt), x_min, y_min, x_max, y_max, cls])
                cnt+=1

    except Exception as e:
        print(i)
        raise e
print(cnt1)

10


In [27]:
df_pred_wp = pd.DataFrame(data, columns=['Name', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])

## Data frames

In [28]:
df_gt.head()

,Name,x_min,y_min,x_max,y_max,class
0,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_0,677.5,505.6,1010.0,897.6,1.0
1,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_1,1107.0,1007.0,1405.0,1519.0,0.0
2,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_2,327.5,1248.8,490.0,1718.4,11.0
3,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_3,363.0,1598.0,607.0,1997.0,9.0
4,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_4,300.0,1777.6,437.5,1902.4,10.0


In [29]:
df_gt_wp.head()

,Name,x_min,y_min,x_max,y_max,class
0,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_0,368.625868,274.963889,549.538194,488.147917,1.0
1,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_1,602.027344,547.905903,764.090712,826.483681,0.0
2,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_2,178.191840,679.143403,266.607639,934.529167,11.0
3,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_3,197.412760,869.467361,330.108941,1086.562153,9.0
4,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_4,163.229167,966.724306,238.042535,1034.595139,10.0


In [30]:
df_pred.head()

,Name,x_min,y_min,x_max,y_max,label
0,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_0,1032.324463,1012.293457,1410.901367,1415.352417,0.0
1,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_1,435.428009,948.709534,756.433960,1316.043335,13.0
2,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_2,394.834259,936.460144,827.138245,1719.329834,13.0
3,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_3,436.574280,688.318848,913.112732,1113.716064,1.0
4,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_4,376.870056,663.337769,946.169922,1376.797852,1.0


In [31]:
df_pred_wp.head()

,Name,x_min,y_min,x_max,y_max,label
0,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_0,234.118698,514.765442,410.596405,716.427612,13.0
1,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_1,564.053406,551.031189,766.893555,765.209656,0.0
2,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_2,207.434494,507.496490,455.350983,939.540527,1.0
3,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_3,208.180389,511.461426,444.322510,938.135437,13.0
4,dc7d6c6fa1fdde25e0aa64a1f6fd594a.png_4,196.941101,356.972443,516.269104,752.030762,1.0


## LI Score

In [32]:
from cheXwhatsApp import li_score

In [33]:
ans = li_score(df_pred, df_pred_wp, df_gt, df_gt_wp, iou_thresholds=[0.1,0.3,0.5,0.7,0.9])

In [34]:
type(ans)

dict

In [35]:
ans[0.5]

0.1

In [36]:
ans

{0.1: 0.125, 0.3: 0.1, 0.5: 0.1, 0.7: 0.075, 0.9: 0.0}